In [20]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Task 2: Merging Datasets with Transitive Properties

In [31]:
# Load necessary datasets
transactions = pd.read_csv('customer_transactions_augmented.csv')
social_profiles = pd.read_csv('customer_social_profiles.csv')
id_mapping = pd.read_csv('id_mapping.csv')

# Inspect data
print("Transactions Sample:")
print(transactions.head())
print("Social Profiles Sample:")
print(social_profiles.head())
print("ID Mapping Sample:")
print(id_mapping.head())

Transactions Sample:
   customer_id_legacy  transaction_id  purchase_amount  customer_rating  \
0                 151            1001       411.072250         2.241585   
1                 192            1002       329.532788         4.305076   
2                 114            1003       430.609797         2.141953   
3                 171            1004       251.203273         2.841900   
4                 160            1005        55.177123         1.355378   

  purchase_date product_category  
0    2024-01-01           Sports  
1    2024-01-02      Electronics  
2    2024-01-03      Electronics  
3    2024-01-04         Clothing  
4    2024-01-05         Clothing  
Social Profiles Sample:
  customer_id_new social_media_platform  engagement_score  \
0            A178              LinkedIn                74   
1            A190               Twitter                82   
2            A150              Facebook                96   
3            A162               Twitter           

In [32]:
# Merge transactions with id_mapping to get the new customer ID
transactions = transactions.merge(id_mapping, left_on='customer_id_legacy', right_on='customer_id_legacy', how='left')

In [33]:
# Merge with social_profiles using the new customer ID
merged_data = transactions.merge(social_profiles, left_on='customer_id_new', right_on='customer_id_new', how='left')

In [34]:
# Handle potential duplicate mappings (one-to-many relationships)
merged_data.drop_duplicates(subset=['customer_id_new'], keep='first', inplace=True)

In [35]:
# Feature Engineering: Customer Engagement Score
merged_data['engagement_score'] = (merged_data['purchase_amount'].fillna(0) * 0.6) + (merged_data['purchase_interest_score'].fillna(0) * 0.4)

In [36]:
# Export final merged dataset
merged_data.to_csv('final_customer_data_groupX.csv', index=False)

In [37]:
# Display summary
print("Final Merged Data Sample:")
print(merged_data.head())

Final Merged Data Sample:
   customer_id_legacy  transaction_id  purchase_amount  customer_rating  \
0                 151            1001       411.072250         2.241585   
1                 192            1002       329.532788         4.305076   
2                 192            1002       329.532788         4.305076   
3                 192            1002       329.532788         4.305076   
4                 192            1002       329.532788         4.305076   

  purchase_date product_category customer_id_new social_media_platform  \
0    2024-01-01           Sports            A176              Facebook   
1    2024-01-02      Electronics            A156             Instagram   
2    2024-01-02      Electronics            A108                   NaN   
3    2024-01-02      Electronics            A110                   NaN   
4    2024-01-02      Electronics            A140              LinkedIn   

   engagement_score  purchase_interest_score review_sentiment  
0        247.9